# k-Nearest Neighbors Classifier (k-NN)

O **k-NN** é o classificador mais simples na área de aprendizado de máquina. Diferentemente das redes neurais, não se realiza de fato um **aprendizado**, em vez disso, o algoritmo verifica a distância entre o objeto a ser classificado e os vetores de característica. Devido a sua simplicidade, é bastante utilizado em *benchmarks* de classificadores mais complexos como, Artificial Neural Network (**ANN**) e Suport Vector Machine (**SVM**).

<center>
<figure>
    <img src="notebook-images/classifier_comparison.png" alt="Classifier Comparison">
    <figcaption>Figura 1 - Comparação entre os classificadores (https://amueller.github.io/applied_ml_spring_2017/images/classifier_comparison.png).</figcaption>
</figure>
</center>

## 1. Introdução

O **k-NN** não passa por um processo de aprendizagem como o **ANN** e **SVM**, contudo existe um mecanismo de *mapeamento* que servirá para criar o modelo utilizado na classificação dos dados. Este modelo consiste numa lista de **descritores**, que tratam-se de listas das características que descrevem os exemplos dos modelos.

<center>
    <img src="notebook-images/descritor1.jpg" width="450" height="450" style="float:left">
    <img src="notebook-images/descritor2.gif"  width="400" height="400" style="float:right">
    <figcaption style="clear:both">Figura 2 - Exemplos de descritores (vetores de características) de imagens.</figcaption>
</center>

A Figura 2 mostra exemplos de vetores de características sobre imagens, onde cada pixel se torna um elemento do vetor. Portanto cada imagem será representado por seu vetor característico correspondente, o qual fará parte de um conjunto de vetores que serão utilizados no processo de classificação. Além desse tipo de vetor característica, os elementos desses vetores podem ser nominais ou reais, como: **cor, preço, ano, altura, peso, estado civil, etc**. Outra informação relevante é que cada vetor possui uma **classe** associada, o qual servirá de referência para classificar os dados. A Figura 3 mostra a forma geral de um conjunto de descritores e suas repectivas classes. 

<center>
    <img src="notebook-images/dataset_description.jpg" width="500"/>
    <figcaption>Figura 3 - Conjunto de descritores genérico.</figcaption>
</center>

### 1.1 Obtenção dos dados

Neste problema será utilizado o *dataset* <a href="http://archive.ics.uci.edu/ml/datasets/Iris">Iris</a>, que consiste em um conjunto de dados que visa classificar os tipos de flores Ísis em **Setosa, Versicolour e Virginica**. Esse dataset é composto por 150 instâncias, sendo 50 para cada classe. Estes descritores são formados por 4 atributos: **tamanho da sépala, largura da sépala, tamanho da pétala** e **largura da pétala**.

In [3]:
import pandas as pd

# Carregando o dataset
dataset = pd.read_csv("dataset/iris/dataset.csv", header=None)

class_column = len (dataset.columns) - 1

# Checando os dados
print (dataset)

# Descobrindo o número de instâncias por classes
for i in pd.unique(dataset[class_column]):
    print( i + ': ' + str(len (dataset.loc[dataset[class_column] == i])) )

       0    1    2    3               4
0    5.1  3.5  1.4  0.2     Iris-setosa
1    4.9  3.0  1.4  0.2     Iris-setosa
2    4.7  3.2  1.3  0.2     Iris-setosa
3    4.6  3.1  1.5  0.2     Iris-setosa
4    5.0  3.6  1.4  0.2     Iris-setosa
5    5.4  3.9  1.7  0.4     Iris-setosa
6    4.6  3.4  1.4  0.3     Iris-setosa
7    5.0  3.4  1.5  0.2     Iris-setosa
8    4.4  2.9  1.4  0.2     Iris-setosa
9    4.9  3.1  1.5  0.1     Iris-setosa
10   5.4  3.7  1.5  0.2     Iris-setosa
11   4.8  3.4  1.6  0.2     Iris-setosa
12   4.8  3.0  1.4  0.1     Iris-setosa
13   4.3  3.0  1.1  0.1     Iris-setosa
14   5.8  4.0  1.2  0.2     Iris-setosa
15   5.7  4.4  1.5  0.4     Iris-setosa
16   5.4  3.9  1.3  0.4     Iris-setosa
17   5.1  3.5  1.4  0.3     Iris-setosa
18   5.7  3.8  1.7  0.3     Iris-setosa
19   5.1  3.8  1.5  0.3     Iris-setosa
20   5.4  3.4  1.7  0.2     Iris-setosa
21   5.1  3.7  1.5  0.4     Iris-setosa
22   4.6  3.6  1.0  0.2     Iris-setosa
23   5.1  3.3  1.7  0.5     Iris-setosa


## 2. Princípio de funcionamento

A ideia básica do classificador **k-NN** está em medir a *distância* entre o indivíduo a ser classificado e os descritores, onde a classe atribuida a esse indivíduo será a mesma que a maioria dos **k** descritores mais próximos. Existem vários cálculos de distâncias que podem ser utilizados como métricas para encontrar os descritores mais próximos, sendo as mais conhecidas a *Distância Euclidiana* e a *Distância Manhattan*. 

<center>
    <img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/795b967db2917cdde7c2da2d1ee327eb673276c0" width="450">
    <figcaption style="clear:both">Equação 1 - Fórmula da Distância Euclidiana.</figcaption>
    <img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/02436c34fc9562eb170e2e2cfddbb3303075b28e"  width="400">
    <figcaption style="clear:both">Equação 2 - Fórmula da Distância Manhattan.</figcaption>
</center>

### 2.1 Criando as funções para cálculos de distâncias

In [4]:
from math import sqrt

def euclidean(p, q):
    if len(p) != len (q):
        return -1
    
    local_sum = 0
    for i in range(0, len(p)):
        local_sum += pow(q[i] - p[i], 2)
    
    return sqrt (local_sum)

def manhattan(p, q):
    if len(p) != len (q):
        return -1
    
    local_sum = 0
    for i in range(0, len(p)):
        local_sum += abs(p[i] - q[i])
    
    return local_sum

## 3. Pré-processamento

O processo geral de implementação do **k-NN** segue as seguintes etapas:

<ul>
    <li>
        <strong>Etapa 1: </strong>
        Obter os dados, assim como verificar a precisão dos dados, realizar correções e remoção de dados desnecessários.
    </li>
    <li>
        <strong>Etapa 2: </strong>
        Separar o conjunto de dados em 2 conjuntos: <i>treino</i> e <i>testes</i>, sendo o primeiro composto por cerca de 60%-85% do total, e o segundo com o restante.
    </li>
    <li>
        <strong>Etapa 3: </strong>
        Realizar a classificação, seguindo o princípio de funcionamento descrito acima.
    </li>
</ul>

In [5]:
import numpy as np

# Embaralhando os dados
dataset = dataset.iloc[np.random.permutation(len(dataset))]

# Separando o dataset por classes
setosa     = dataset.loc[dataset[class_column] == 'Iris-setosa']
versicolor = dataset.loc[dataset[class_column] == 'Iris-versicolor']
virginica  = dataset.loc[dataset[class_column] == 'Iris-virginica']

In [6]:
train_percentage = 0.75

# Obtendo os conjuntos de treino e de testes
trainset = pd.concat([    setosa[0: int (len(setosa)     * train_percentage + 1)],
                      versicolor[0: int (len(versicolor) * train_percentage + 1)],
                       virginica[0: int (len(virginica)  * train_percentage + 1)]])

testset =  pd.concat([    setosa[int (len(setosa)     * train_percentage + 1) : ],
                      versicolor[int (len(versicolor) * train_percentage + 1) : ],
                       virginica[int (len(virginica)  * train_percentage + 1) : ]])


In [8]:
from operator import itemgetter
from collections import Counter

# Criando a função de classificação
def knn(k, trainset, element):
    distance = []
    
    for _, row in trainset.iterrows():
        distance.append((manhattan(row[0:class_column], element[0:class_column]), row[class_column]))
    
    distance.sort(key=itemgetter(0))
    distance = [classes[1] for classes in distance[0:k]]
    
    most_common = Counter(distance)
    #print("Classification: " + max(most_common, key=most_common.get) + ", " + element[4])
    return max(most_common, key=most_common.get)

# Função de avaliação de acurácia
def evaluate(k, trainset, testset):
    acc = 0
    for _, row in testset.iterrows():
        if( knn(k, trainset, row) == row[class_column] ):
            acc += 1
    
    return acc / len(testset)

# Descobrindo a acurácia para todas as configurações possíveis
def evaluate_by_config(trainset, testset):
    for k in range(1,len(dataset) + 1):
        print("K = " + str(k) + ", acc = " + str(evaluate(k, trainset, testset)))
        
evaluate_by_config(trainset, testset)

K = 1, acc = 0.9722222222222222
K = 2, acc = 0.9722222222222222
K = 3, acc = 1.0
K = 4, acc = 0.9722222222222222
K = 5, acc = 1.0
K = 6, acc = 0.9722222222222222
K = 7, acc = 0.9722222222222222
K = 8, acc = 0.9722222222222222
K = 9, acc = 1.0
K = 10, acc = 0.9722222222222222
K = 11, acc = 0.9722222222222222
K = 12, acc = 0.9722222222222222
K = 13, acc = 0.9722222222222222
K = 14, acc = 0.9722222222222222
K = 15, acc = 0.9722222222222222
K = 16, acc = 0.9722222222222222
K = 17, acc = 0.9722222222222222
K = 18, acc = 0.9722222222222222
K = 19, acc = 1.0
K = 20, acc = 0.9722222222222222
K = 21, acc = 0.9722222222222222
K = 22, acc = 0.9722222222222222
K = 23, acc = 0.9444444444444444
K = 24, acc = 0.9722222222222222
K = 25, acc = 0.9444444444444444
K = 26, acc = 0.9444444444444444
K = 27, acc = 0.9444444444444444
K = 28, acc = 0.9444444444444444
K = 29, acc = 0.9166666666666666
K = 30, acc = 0.9166666666666666
K = 31, acc = 0.9166666666666666
K = 32, acc = 0.9444444444444444
K = 33, acc =

## Referências

[1] Kevin Zakka. A Complete Guide to K-Nearest-Neighbors with Applications in Python and R, Available at: https://kevinzakka.github.io/2016/07/13/k-nearest-neighbor/ (Accessed: 28th March 2018).

[2] Maxwell. Aprendizado de máquina - conceitos básicos, Available at: https://www.maxwell.vrac.puc-rio.br/25796/25796_4.PDF (Accessed: 28th March 2018).

[3] Wikipedia (24th February 2018) k-nearest neighbors algorithm, Available at: https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm (Accessed: 28th March 2018).